## 提前停止

### 加载模型

In [1]:
from transformers import AutoModel, AutoTokenizer

In [2]:
model_dir = '/root/.cache/huggingface/hub/THUDM/chatglm2-6b-32k'
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
model = AutoModel.from_pretrained(model_dir, trust_remote_code=True).quantize(8).half().cuda()

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [6]:
model = model.eval()

In [3]:
history = []
response, history = model.chat(tokenizer, "晚上睡不着应该怎么办", history=history)

In [4]:
print(response)

以下是一些有助于晚上入睡的技巧:

1. 制定规律的睡眠时间表:保持规律的睡眠时间表可以帮助身体调整并期待特定的睡眠时间。

2. 创造一个舒适的睡眠环境:确保卧室安静、黑暗、凉爽、舒适,床垫、枕头舒适,保证睡眠环境符合个人喜好。

3. 避免刺激性物质:避免饮用含咖啡因的饮料、避免喝酒、避免吸烟等刺激性物质,这些物质会影响睡眠质量。

4. 放松身体:使用放松技巧,如深呼吸、渐进性肌肉松弛、瑜伽等,帮助身体放松并减少压力。

5. 避免在床上使用电子产品:避免在床上使用电子产品,如手机、平板电脑等,这些设备会发出蓝光,影响身体放松和睡眠。

6. 远离刺激性活动:避免在睡前进行刺激性活动,如剧烈运动、看紧张刺激的电影等。

7. 采用睡前习惯:采用一些有助于入睡的睡前习惯,如读书、听轻柔的音乐、洗澡等。

如果这些方法不能解决您的问题,可以尝试寻求专业帮助。


### Stop Criteria & 停止标准

> 假设我不需要这么多的回答， 我希望提前停止

In [13]:
from transformers.generation.stopping_criteria import StoppingCriteria, StoppingCriteriaList, \
                        STOPPING_CRITERIA_INPUTS_DOCSTRING, add_start_docstrings
import torch

from typing import List

In [7]:
print(STOPPING_CRITERIA_INPUTS_DOCSTRING)


    Args:
        input_ids (`torch.LongTensor` of shape `(batch_size, sequence_length)`):
            Indices of input sequence tokens in the vocabulary.

            Indices can be obtained using [`BertTokenizer`]. See [`PreTrainedTokenizer.encode`] and
            [`PreTrainedTokenizer.__call__`] for details.

            [What are input IDs?](../glossary#input-ids)
        scores (`torch.FloatTensor` of shape `(batch_size, config.vocab_size)`):
            Prediction scores of a language modeling head. These can be scores for each vocabulary token before SoftMax
            or scores for each vocabulary token after SoftMax.
        kwargs:
            Additional stopping criteria specific kwargs.

    Return:
        `bool`. `False` indicates we should continue, `True` indicates we should stop.




> * `StoppingCriteriaList` 是一个容器， 需要将所有的`criteria`都添加到其中，generate时传入的是这个容器.<br>
> *  `StoppingCriteria`是基础类， 自定义的`critieria`继承这个基础类

### 自定义Criteria

In [14]:
class StopAtSpecificTokenCriteria(StoppingCriteria):
    """
    当生成第一个指定Token时，立即停止生成
    """
    
    def __init__(self, token_id_list: List[int] = None):
        self.token_id_list = token_id_list
        
    @add_start_docstrings(STOPPING_CRITERIA_INPUTS_DOCSTRING)
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        # return np.argmax(scores[-1].detach().cpu().numpy() in slef.token_id_list
        # 存储scores会额外占用资源，所以直接用input_ids进行判断
        return input_ids[0][-1].detach().cpu().numpy() in self.token_id_list

In [9]:
tokenizer.convert_tokens_to_ids([':'])

[30954]

In [11]:
tokenizer.convert_ids_to_tokens([30954])

[':']

In [15]:
stopping_criteria = StoppingCriteriaList()
stopping_criteria.append(StopAtSpecificTokenCriteria(token_id_list=[30954]))

In [17]:
history = []
response, history = model.chat(tokenizer, "晚上睡不着应该怎么办", history=history, stopping_criteria=stopping_criteria)

Generate config GenerationConfig {
  "_from_model_config": true,
  "eos_token_id": 2,
  "pad_token_id": 0,
  "transformers_version": "4.30.2"
}



In [18]:
print(response)

以下时是一些有助于晚上入睡的方法:


### 结论

> 模型遇到":" 就不再继续浪费资源生成了。

In [19]:
from IPython.display import display, clear_output
history = []
for reponse, history in model.stream_chat(tokenizer=tokenizer, query="晚上睡不着应该怎么办", history=history, stopping_criteria=stopping_criteria):
    clear_output(wait=True)
    print(reponse)

晚上睡不着觉可能会让人感到非常不舒服,可以尝试以下一些方法来帮助入睡:
